In [0]:
import requests
import json
import pandas as pd

from pyspark.sql.functions import col, current_timestamp, lit, to_json

In [0]:
url_mercado = "https://api.cartola.globo.com/atletas/mercado"
mercado = requests.get(url_mercado).json()

rodada_atual = mercado.get("rodada_atual")
status_mercado = mercado.get("status_mercado")

print(f"Rodada referência: {rodada_atual}")
print(f"Status mercado: {status_mercado}")

In [0]:
print(mercado.keys())

In [0]:
# tabela clubes
clubes = mercado.get("clubes", {})

print("Quantidade de clubes:", len(clubes))

if clubes:
    
    lista_clubes = []
    
    for clube_id, dados in clubes.items():
        dados["clube_id"] = int(clube_id)
        lista_clubes.append(dados)
    
    df_clubes = spark.createDataFrame(pd.DataFrame(lista_clubes)) \
        .withColumn("rodada_referencia", lit(rodada_atual)) \
        .withColumn("dt_ingestao", current_timestamp())
    
    df_clubes.write \
        .format("delta") \
        .mode("append") \
        .saveAsTable("project_data_football_bronze.clubes")
    
    print("Clubes carregados com sucesso!")

In [0]:
# tabela posicoes
posicoes = mercado.get("posicoes", {})

print("Quantidade de posicoes:", len(posicoes))

if posicoes:
    
    lista_posicoes = []
    
    for pos_id, dados in posicoes.items():
        dados["posicao_id"] = int(pos_id)
        lista_posicoes.append(dados)
    
    df_posicoes = spark.createDataFrame(pd.DataFrame(lista_posicoes)) \
        .withColumn("rodada_referencia", lit(rodada_atual)) \
        .withColumn("dt_ingestao", current_timestamp())
    
    df_posicoes.write \
        .format("delta") \
        .mode("append") \
        .saveAsTable("project_data_football_bronze.posicoes")
    
    print("Posições carregadas com sucesso!")

In [0]:
# tabela atletas

atletas = mercado.get("atletas", {})

print("Quantidade de atletas:", len(atletas))

if atletas and len(atletas) > 0:

    lista_atletas = [dict(a) for a in atletas]
    
    df_atletas = spark.createDataFrame(pd.DataFrame(lista_atletas)) \
        .withColumn("rodada_referencia", lit(rodada_atual)) \
        .withColumn("dt_ingestao", current_timestamp())
    
    df_atletas.write \
        .format("delta") \
        .mode("append") \
        .saveAsTable("project_data_football_bronze.atletas")
    
    print("Atletas carregados com sucesso!")